<a href="https://colab.research.google.com/github/itsamekadio/Numerical_Methods_Course/blob/main/gauss_w_its_look_alikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np

def forward_elimination(A, b):
    n = len(A)
    for k in range(n - 1):
        for i in range(k + 1, n):
            factor = A[i][k] / A[k][k]
            for j in range(k, n):
                A[i][j] -= factor * A[k][j]
            b[i] -= factor * b[k]

def back_substitution(A, b):
    n = len(A)
    x = np.zeros(n)
    x[n - 1] = b[n - 1] / A[n - 1][n - 1]
    for i in range(n - 2, -1, -1):
        sum_ = b[i]
        for j in range(i + 1, n):
            sum_ -= A[i][j] * x[j]
        x[i] = sum_ / A[i][i]
    return x

def pivot(A, b, k):
    n = len(A)
    max_row = k + np.argmax(np.abs(A[k:n, k]))
    if max_row != k:
        A[[k, max_row]] = A[[max_row, k]]
        b[[k, max_row]] = b[[max_row, k]]

def full_elimination(A, b):
    n = len(A)
    for k in range(n):
        pivot(A, b, k)


        factor = A[k, k]
        A[k] /= factor
        b[k] /= factor


        for i in range(n):
            if i != k:
                factor = A[i, k]
                A[i] -= factor * A[k]
                b[i] -= factor * b[k]

def gauss_jordan(A, b):
    full_elimination(A, b)
    return b
def gauss(A, b, tol=1e-9):

    n = len(A)

    for k in range(n - 1):
        pivot(A, b, k)

    forward_elimination(A, b)
    return back_substitution(A, b)
def naive_gauss_elimination(A, b):
    forward_elimination(A, b)
    return back_substitution(A, b)
def matrix_norm(A, norm_type='inf'):
    if norm_type == 'inf':
        return np.linalg.norm(A, ord=np.inf)
    elif norm_type == 'fro':
        return np.linalg.norm(A, ord='fro')



def condition_number(A, norm_type='inf'):
    norm_A = matrix_norm(A, norm_type)
    norm_A_inv = matrix_norm(np.linalg.inv(A), norm_type)
    cond_number = norm_A * norm_A_inv
    print(cond_number)
    return cond_number

def is_well_conditioned(A, norm_type='inf', threshold=100):
    cond_num = condition_number(A, norm_type)
    return "Well-conditioned" if cond_num < threshold else "Ill-conditioned"


A = np.array([[1.0, 0.5, 1/3],
              [0.5, 1/3.0, .25],
              [1/3, .25, 1/5]])

b = np.array([7.85, -19.3, 71.4])
print("Matrix Condition:", is_well_conditioned(A, 'inf'))
solution = gauss_jordan(A, b)
print("Solution:", solution)
print(A)



748.0000000000027
Matrix Condition: Ill-conditioned
Solution: [  2907.45 -16840.2   16561.5 ]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0. -0.  1.]]


In [14]:
import numpy as np

def is_diagonally_dominant(A):
    n = len(A)
    for i in range(n):
        diag = abs(A[i][i])
        off_diag_sum = sum(abs(A[i][j]) for j in range(n) if i != j)
        if diag < off_diag_sum:
            return False
    return True

def gauss_seidel(A, b, x0, max_iter, epsilon, relaxation=False, lambd=1.0):
    if not is_diagonally_dominant(A):
        print("Matrix is not diagonally dominant")
        return None

    n = len(b)
    x = np.array(x0, dtype=float)

    for _ in range(max_iter):
        for i in range(n):
            old = x[i]
            sum_ = b[i]
            for j in range(n):
                if i != j:
                    sum_ -= A[i][j] * x[j]
            sum_ /= A[i][i]

            x[i] = lambd * sum_ + (1 - lambd) * old if relaxation else sum_

            if x[i] != 0 and abs((x[i] - old) / x[i]) * 100 <= epsilon:
                return x

    return x
